In [316]:
import pandas as pd
import numpy as np
import nltk
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [317]:
df_analyze = pd.read_csv('data/payments_training.tsv',index_col=False,sep='\t', names = ['id','date','payment','text'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_2372\2288680057.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_analyze = pd.read_csv('data/payments_training.tsv',index_col=False,sep='\t', names = ['id','date','payment','text'])


In [318]:
df_analyze.head()

,id,date,payment,text
0,1,07.11.2024,15300.00,За участие в конференции в г. Майкоп по догово...
1,2,07.11.2024,"40200,00",За оказание услуг по договору №79-02726В от 01...
2,3,07.11.2024,1440-00,Оплата за Порошок стиральный Ariel Color autom...
3,4,07.11.2024,240000000-00,Возврат денежных средств по договору займа №04...
4,5,07.11.2024,1360000.00,"Оплата Дог №452 от 13/03/2021, согл. Сч 0745-2..."


In [319]:
df_analyze = df_analyze.drop(columns = ['id','date'])

In [320]:
df_analyze.head()

,payment,text
0,15300.00,За участие в конференции в г. Майкоп по догово...
1,"40200,00",За оказание услуг по договору №79-02726В от 01...
2,1440-00,Оплата за Порошок стиральный Ariel Color autom...
3,240000000-00,Возврат денежных средств по договору займа №04...
4,1360000.00,"Оплата Дог №452 от 13/03/2021, согл. Сч 0745-2..."


#### Step-1: Remove non-cyrillic symbols

In [321]:
def remove_non_cyrillic(text):
 cyrillic_pattern = r"[^а-яА-ЯёЁ]+"
 cleaned_text = re.sub(cyrillic_pattern, " ", text)
 return cleaned_text

In [322]:
df_analyze['text'] = df_analyze['text'].apply(remove_non_cyrillic)

In [323]:
df_analyze.head()

,payment,text
0,15300.00,За участие в конференции в г Майкоп по договор...
1,"40200,00",За оказание услуг по договору В от г
2,1440-00,Оплата за Порошок стиральный кг по счету от ав...
3,240000000-00,Возврат денежных средств по договору займа А о...
4,1360000.00,Оплата Дог от согл Сч от г оплата за сброс заг...


#### Step-1.5 Remove joined words

def split_joined_words(text):
    splited_text = text.split(' ')
    tokens = []
    splited = False
    for token in splited_text:
        splited = False
        for i in range(1, len(token)):
            token = token.strip()
            if not token[i-1].isupper() and token[i].isupper():
                token1,token2 = token[0:i], token[i:]
                splited = True
                print(token1,token2)
                tokens.append(f' {token1} ')
                tokens.append(f' {token2} ')
        if splited is False:
            tokens.append(f' {token} ')
    return ''.join(tokens)

In [324]:
def split_joined_words(text):
    
    tokens = []
    
    def split_token(split_points, text):           
        for i in range(1,len(split_points)):
            token = text[split_points[i-1]:split_points[i]]
            tokens.append(f' {token} ')
    
    splited_text = text.split(' ')
    for token in splited_text:
        split_points = [0]
        for i in range(1, len(token)):
            token = token.strip()
            if not token[i-1].isupper() and token[i].isupper():
                split_points.append(i)
        split_points.append(len(token))
        split_token(split_points,token)
                
    return ''.join(tokens)

In [325]:
df_analyze['text'] = df_analyze['text'].apply(split_joined_words)

#### Step-2: Apply stop words and lemmatize

In [326]:
stopwords_ru = stopwords.words("russian")
stopwords_ru.extend( {"по", "от", "до", "с", "в", "на", "за", "к", "о", "об", "у", "со", "из", "при", "под", "про", "через", "над", "без"})
morph = MorphAnalyzer()

In [327]:
def remove_stop_words(text):
    text_splited = text.split(' ')
    filtered_tokens = []
    for token in text_splited:
        if token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            if len(token) < 2: # Отсев единичных букв
                continue
            filtered_tokens.append(f' {token} ')
    # if len(filtered_tokens) <= 2:
    #    return None # ( ' ' ?)
    return ''.join(filtered_tokens)

In [328]:
df_analyze['text'] = df_analyze['text'].apply(remove_stop_words)

In [329]:
df_analyze.head()

,payment,text
0,15300.00,за участие конференция майкоп договор дв...
1,"40200,00",за оказание услуга договор
2,1440-00,оплата порошок стиральный кг счёт август...
3,240000000-00,возврат денежный средство договор заём б...
4,1360000.00,оплата дог соглый сч оплата сброс загря...


#### Step-3: TF-IDF

In [330]:
texts = df_analyze['text']

In [331]:
texts_raw = texts.values
unique_tokens = []
for text in texts_raw:
    tokens = text.split(' ')
    for token in tokens:
        if token not in unique_tokens:
            unique_tokens.append(token)

count_unique_tokens = len(unique_tokens)
print(f' Уникальных токенов: {count_unique_tokens}')

 Уникальных токенов: 563


In [332]:
tfidf = TfidfVectorizer(max_features=count_unique_tokens) 

In [333]:
tfidf_matrix = tfidf.fit_transform(texts).toarray()

#### Step-4: PCA

In [334]:
pca_transform = PCA(n_components=199) # ДЛЯ МАЛЕНЬКОГО ДАТАСЕТА!
#pca_transform = PCA(n_components = 744) # ДЛЯ БОЛЬШОГО ДАТАСЕТА

In [335]:
tfidf_matrix_compressed = pca_transform.fit_transform(tfidf_matrix)

In [336]:
print(f'Компонент до сжатия: {tfidf_matrix.shape[1]}, компонент после сжатия: {tfidf_matrix_compressed.shape[1]}')

Компонент до сжатия: 562, компонент после сжатия: 199


In [337]:
explained_variance = np.sum(pca_transform.explained_variance_ratio_)
print(f'Величина объясненной дисперсии: {explained_variance }')

Величина объясненной дисперсии: 0.9903514300631601


In [338]:
tfidf_matrix.shape[0]

500